In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
# Enable automatic conversion between Pandas dataframes and R data frames
# pandas2ri.activate()

In [3]:
# Initialize R instance
r = robjects.r

In [4]:
# Install the GeoLift package
r('''
if (!require("GeoLift")) {
    install.packages("GeoLift", repos="http://cran.us.r-project.org")
    library(GeoLift)
}
''')

R[write to console]: Loading required package: GeoLift



In [25]:
r('''
data(GeoLift_PreTest)
''')

In [26]:
r('''
GeoTestData_PreTest <- GeoDataRead(data = GeoLift_PreTest,
                                        date_id = "date",
                                        location_id = "location",
                                        Y_id = "Y",
                                        X = c(), #empty list as we have no covariates
                                        format = "yyyy-mm-dd",
                                        summary = TRUE)
''')

R[write to console]: ##################################
#####       Summary       #####
##################################

* Raw Number of Locations: 40
* Time Periods: 90
* Final Number of Locations (Complete): 40



In [27]:
# # Step 1: Load Data
# with localconverter(robjects.default_converter + pandas2ri.converter):
#     geo_data = robjects.conversion.py2rpy(pd.read_csv('gt-sample.csv'))

#     r.assign('geo_data', geo_data)

In [28]:
# r('''
# GeoTestData_PreTest <- GeoDataRead(data = geo_data,
#                                         date_id = "date",
#                                         location_id = "location",
#                                         Y_id = "subs",
#                                         X = c(), #empty list as we have no covariates
#                                         format = "yyyy-mm-dd",
#                                         summary = TRUE)
# ''')

In [ ]:
# Step 2: Pre-Analysis
# Replace this with the appropriate pre-analysis code from the walkthrough
r('''
# Your pre-analysis code here
''')

# Step 3: Model Diagnostics
# Replace with model diagnostic code from the walkthrough
r('''
# Your model diagnostics code here
''')

# Step 4: Post-Analysis
# Replace with post-analysis code from the walkthrough
r('''
# Your post-analysis code here
''')

# Step 5: View Results
# Replace with the code to view results from the walkthrough
r('''
# Your code to view results here
''')
